In [ ]:
# Source: https://leetcode.com/problems/find-consistently-improving-employees/solutions/6863954/simple-best-solution-by-iqbaldiit-z17l/
'''
Table: employees

	+-------------+---------+
	| Column Name | Type    |
	+-------------+---------+
	| employee_id | int     |
	| name        | varchar |
	+-------------+---------+
	employee_id is the unique identifier for this table.
	Each row contains information about an employee.
	Table: performance_reviews

	+-------------+------+
	| Column Name | Type |
	+-------------+------+
	| review_id   | int  |
	| employee_id | int  |
	| review_date | date |
	| rating      | int  |
	+-------------+------+
	review_id is the unique identifier for this table.
	Each row represents a performance review for an employee. The rating is on a scale of 1-5 where 5 is excellent and 1 is poor.
	Write a solution to find employees who have consistently improved their performance over their last three reviews.

	An employee must have at least 3 review to be considered
	The employee's last 3 reviews must show strictly increasing ratings (each review better than the previous)
	Use the most recent 3 reviews based on review_date for each employee
	Calculate the improvement score as the difference between the latest rating and the earliest rating among the last 3 reviews
	Return the result table ordered by improvement score in descending order, then by name in ascending order.

	The result format is in the following example.

 

	Example:

	Input:

	employees table:

	+-------------+----------------+
	| employee_id | name           |
	+-------------+----------------+
	| 1           | Alice Johnson  |
	| 2           | Bob Smith      |
	| 3           | Carol Davis    |
	| 4           | David Wilson   |
	| 5           | Emma Brown     |
	+-------------+----------------+
	performance_reviews table:

	+-----------+-------------+-------------+--------+
	| review_id | employee_id | review_date | rating |
	+-----------+-------------+-------------+--------+
	| 1         | 1           | 2023-01-15  | 2      |
	| 2         | 1           | 2023-04-15  | 3      |
	| 3         | 1           | 2023-07-15  | 4      |
	| 4         | 1           | 2023-10-15  | 5      |
	| 5         | 2           | 2023-02-01  | 3      |
	| 6         | 2           | 2023-05-01  | 2      |
	| 7         | 2           | 2023-08-01  | 4      |
	| 8         | 2           | 2023-11-01  | 5      |
	| 9         | 3           | 2023-03-10  | 1      |
	| 10        | 3           | 2023-06-10  | 2      |
	| 11        | 3           | 2023-09-10  | 3      |
	| 12        | 3           | 2023-12-10  | 4      |
	| 13        | 4           | 2023-01-20  | 4      |
	| 14        | 4           | 2023-04-20  | 4      |
	| 15        | 4           | 2023-07-20  | 4      |
	| 16        | 5           | 2023-02-15  | 3      |
	| 17        | 5           | 2023-05-15  | 2      |
	+-----------+-------------+-------------+--------+
	Output:

	+-------------+----------------+-------------------+
	| employee_id | name           | improvement_score |
	+-------------+----------------+-------------------+
	| 2           | Bob Smith      | 3                 |
	| 1           | Alice Johnson  | 2                 |
	| 3           | Carol Davis    | 2                 |
	+-------------+----------------+-------------------+
	Explanation:

	Alice Johnson (employee_id = 1):
	Has 4 reviews with ratings: 2, 3, 4, 5
	Last 3 reviews (by date): 2023-04-15 (3), 2023-07-15 (4), 2023-10-15 (5)
	Ratings are strictly increasing: 3 → 4 → 5
	Improvement score: 5 - 3 = 2
	Carol Davis (employee_id = 3):
	Has 4 reviews with ratings: 1, 2, 3, 4
	Last 3 reviews (by date): 2023-06-10 (2), 2023-09-10 (3), 2023-12-10 (4)
	Ratings are strictly increasing: 2 → 3 → 4
	Improvement score: 4 - 2 = 2
	Bob Smith (employee_id = 2):
	Has 4 reviews with ratings: 3, 2, 4, 5
	Last 3 reviews (by date): 2023-05-01 (2), 2023-08-01 (4), 2023-11-01 (5)
	Ratings are strictly increasing: 2 → 4 → 5
	Improvement score: 5 - 2 = 3
	Employees not included:
	David Wilson (employee_id = 4): Last 3 reviews are all 4 (no improvement)
	Emma Brown (employee_id = 5): Only has 2 reviews (needs at least 3)
	The output table is ordered by improvement_score in descending order, then by name in ascending order.
'''

In [37]:
import pandas as pd

def find_consistently_improving_employees(employees: pd.DataFrame, performance_reviews: pd.DataFrame) -> pd.DataFrame:
    #get the most recent reviews first
    performance_reviews=performance_reviews.sort_values(by=['employee_id','review_date'],ascending=[True,False])

    #get last three reviews
    performance_reviews=performance_reviews.groupby("employee_id").head(3)

    performance_reviews["2nd_rating"]=performance_reviews.groupby("employee_id")["rating"].shift(1)
    performance_reviews["last_rating"]=performance_reviews.groupby("employee_id")["rating"].shift(2)
    performance_reviews=performance_reviews.dropna()
    performance_reviews=performance_reviews[(performance_reviews["last_rating"]>performance_reviews["2nd_rating"]) & (performance_reviews["2nd_rating"]>performance_reviews["rating"])]
    performance_reviews["improvement_score"]=(performance_reviews["last_rating"]-performance_reviews["rating"]).astype(int)

    df=(performance_reviews.merge(employees,on="employee_id")
        .reset_index()[["employee_id","name","improvement_score"]]
        .sort_values(['improvement_score','name'],ascending=[False,True]))
    return df    

In [38]:
# Sample Employees Data
employees_data = {
    "employee_id": [1, 2, 3, 4, 5],
    "name": ["Alice Johnson", "Bob Smith", "Carol Davis", "David Wilson", "Emma Brown"]
}

# Sample Performance Reviews Data
reviews_data = {
    "review_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
    "employee_id": [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5],
    "review_date": pd.to_datetime([
        "2023-01-15", "2023-04-15", "2023-07-15", "2023-10-15",
        "2023-02-01", "2023-05-01", "2023-08-01", "2023-11-01",
        "2023-03-10", "2023-06-10", "2023-09-10", "2023-12-10",
        "2023-01-20", "2023-04-20", "2023-07-20",
        "2023-02-15", "2023-05-15"
    ]),
    "rating": [2, 3, 4, 5, 3, 2, 4, 5, 1, 2, 3, 4, 4, 4, 4, 3, 2]
}

# Create DataFrames
employees = pd.DataFrame(employees_data)
performance_reviews = pd.DataFrame(reviews_data)

#Invoke function
find_consistently_improving_employees(employees,performance_reviews)

,employee_id,name,improvement_score
1,2,Bob Smith,3
0,1,Alice Johnson,2
2,3,Carol Davis,2
